In [1]:
# Generate the video for the env with fake action

In [2]:
import sys
sys.path.insert(0, "..")

In [3]:
import torch
import gymnasium as gym

In [4]:
import torchdriveenv
from torchdriveenv.env_utils import load_waypoint_suite_data, load_rl_training_config, load_labeled_data

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.
2024-11-24 20:49:50.610138: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 20:49:50.617135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 20:49:50.625034: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register fa

In [5]:
# data = load_waypoint_suite_data("data/validation_cases.yml")
data = load_labeled_data("/home/kezhang/work/fall_2024/energy-based-diffusion-model/labeled_data")
env_config = load_rl_training_config("env_configs/rl_training.yml").env

In [6]:
env_config.render_mode="video"

In [7]:
env = gym.make('torchdriveenv-v0', args={'cfg': env_config, 'data': data})

INFO:torchdriveenv.gym_env:seed: 2537088724
INFO:torchdriveenv.gym_env:    def get_reward(self):
        x = self.simulator.get_state()[..., 0]
        y = self.simulator.get_state()[..., 1]
        psi = self.simulator.get_state()[..., 2]

        d = math.dist((x, y), (self.last_x, self.last_y)) if (self.last_x is not None) and (self.last_y is not None) else 0
        distance_reward = 1 if d > 0.1 else 0
        psi_reward = (1 - math.cos(psi - self.last_psi)) * (-20.0) if (self.last_psi is not None) else 0
#        speed_reward = -1 if d > 0.5 else 0
        if self.check_reach_target():
            reach_target_reward = 10
            self.reached_waypoint_num += 1
        else:
            reach_target_reward = 0
        r = torch.zeros_like(x)
        r += reach_target_reward + distance_reward + psi_reward
#        r += reach_target_reward + psi_reward
#        r += distance_reward
        return r



In [8]:
def main(cfg, env):
    env.reset()
    while True:
        actions = torch.tensor([1, 0], dtype=torch.float32, device='cpu')  # accelerate hard without steering
        obs, reward, terminated, truncated, info = env.step(actions)  
        if terminated or truncated:
            break
    env.close()

In [9]:
main(env_config, env)

waypoints
[(50.967484397485215, -4.275350231478027), (25.683725525802057, -4.556280885607824), (14.727430014739305, -12.141408547112821), (-0.4428253082705922, -19.72653620861772), (-14.489358014761308, -13.546061817761803), (-20.669832405617214, -0.6232517277903769), (-16.73680324779977, 12.018627708051152), (-9.994467548684263, 23.2558538732437), (-7.1851610073861, 43.20193031646048)]


../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [364,0,0], thread: [118,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [364,0,0], thread: [119,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [364,0,0], thread: [120,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [364,0,0], thread: [121,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator(): block: [364,0,0], thread: [122,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:144: operator

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### 